# 惠聚电商平台用户品类偏好与消费行为深度分析

## 阶段四：购物篮分析 (Market Basket Analysis)

### 1. 引言与目标

在前三个阶段，我们对用户画像、品类受欢迎程度、消费能力以及不同用户群体对品类的偏好进行了深入分析。现在，我们将进行购物篮分析，旨在发现不同商品品类之间共同购买的关联性。

**本阶段目标**：
1.  识别哪些商品品类经常被用户同时购买（频繁项集）。
2.  挖掘品类之间的强关联规则 (e.g., "购买了品类A的用户，有多大概率也会购买品类B?")。
3.  基于挖掘出的关联规则，为平台的交叉销售、捆绑销售、商品推荐、库存管理和营销活动提供数据驱动的建议。

In [1]:
!pip install mlxtend

  Using cached mlxtend-0.23.4-py3-none-any.whl.metadata (7.3 kB)
Using cached mlxtend-0.23.4-py3-none-any.whl (1.4 MB)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 导入购物篮分析相关的库
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder # 用于将数据转换为适合的格式

# --- 绘图美化与中文支持配置 (与Notebook 02/03一致) ---
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('pastel')
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS'] # 替换为你系统中可用的中文字体
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 18
# --- 结束配置 ---

# 配置pandas显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x) # 关联规则的指标可能小数位较多

# 加载数据 (与Notebook 02/03一致，重新加载并进行类型转换)
data_path = '../data/huiju_sales_data_2022_processed.csv'
try:
    df = pd.read_csv(data_path)
    print("原始数据加载成功！")
    
    # 保留需要的列，并确保 Product_Category 是字符串类型，方便后续处理
    # 我们只需要 User_ID 和 Product_Category 来构建事务
    df_mba = df[['User_ID', 'Product_Category']].copy()
    df_mba['Product_Category'] = df_mba['Product_Category'].astype(str) # 转换为字符串，以作为离散项处理
    
    print(f"用于购物篮分析的数据已准备，共 {len(df_mba)} 条购买记录。")
    print(df_mba.head())

except FileNotFoundError:
    print(f"错误：数据文件未在指定路径找到: {data_path}")
    df_mba = None
except Exception as e:
    print(f"加载数据或初步处理时发生错误: {e}")
    df_mba = None

if df_mba is not None:
    print(f"\n独立用户数: {df_mba['User_ID'].nunique()}")
    print(f"独立商品品类数: {df_mba['Product_Category'].nunique()}")

原始数据加载成功！
用于购物篮分析的数据已准备，共 550068 条购买记录。
   User_ID Product_Category
0  1000001                3
1  1000001                1
2  1000001               12
3  1000001               12
4  1000002                8

独立用户数: 5891
独立商品品类数: 20


### 2. 数据准备：构建事务数据

购物篮分析算法（如Apriori）通常需要输入“事务”列表。一个事务代表一次购买行为中包含的商品集合。在我们的案例中，由于原始数据是每行一条购买记录（一个用户可能购买了多个品类，但分散在多行），我们需要将每个用户所有购买过的 **不同商品品类** 汇总起来，形成该用户的一个“事务”。

例如，如果用户A购买了品类1、品类5、品类1，那么他/她的事务是 `{品类1, 品类5}`。

我们将：
1.  按 `User_ID` 分组。
2.  对每个用户，获取其购买过的所有 `Product_Category` 的唯一值列表。
3.  将这个列表数据转换为Apriori算法要求的one-hot编码格式。

In [3]:
if df_mba is not None:
    # 按User_ID分组，将每个用户购买的所有Product_Category汇总成列表
    # 使用 set 去除同一用户购买同一品类的重复记录，确保每个品类在事务中只出现一次
    transactions_grouped = df_mba.groupby('User_ID')['Product_Category'].apply(lambda x: list(set(x))).reset_index()
    
    # 查看转换后的部分数据
    print("按用户分组后的事务列表（部分）：")
    display(transactions_grouped.head())
    
    # 将Product_Category列表提取出来，作为后续编码的输入
    transactions_list = transactions_grouped['Product_Category'].tolist()
    
    # 使用TransactionEncoder将事务列表转换为适合Apriori算法的 one-hot 编码矩阵
    # TransactionEncoder 会自动处理所有出现过的品类，并生成对应的列
    te = TransactionEncoder()
    te_ary = te.fit(transactions_list).transform(transactions_list)
    
    # 将one-hot编码后的数组转换回DataFrame，列名为商品品类ID
    df_onehot = pd.DataFrame(te_ary, columns=te.columns_)
    
    print("\nOne-hot 编码后的事务数据 (部分，显示每个用户是否购买了对应品类):")
    display(df_onehot.head())
    print(f"\nOne-hot 编码后的数据维度: {df_onehot.shape}")
    print(f"列名 (商品品类): {df_onehot.columns.tolist()}")

else:
    print("df_mba 未成功加载，无法进行后续处理。")


按用户分组后的事务列表（部分）：


,User_ID,Product_Category
0,1000001,"[8, 16, 20, 4, 14, 12, 2, 5, 3, 6, 1]"
1,1000002,"[8, 20, 2, 5, 1, 6]"
2,1000003,"[8, 18, 2, 5, 3, 1]"
3,1000004,"[1, 20]"
4,1000005,"[8, 16, 4, 15, 3, 14, 11, 2, 5, 1, 6, 7]"



One-hot 编码后的事务数据 (部分，显示每个用户是否购买了对应品类):


,1,10,11,12,13,14,15,16,17,18,19,2,20,3,4,5,6,7,8,9
0,True,False,False,True,False,True,False,True,False,False,False,True,True,True,True,True,True,False,True,False
1,True,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,True,False,True,False
2,True,False,False,False,False,False,False,False,False,True,False,True,False,True,False,True,False,False,True,False
3,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
4,True,False,True,False,False,True,True,True,False,False,False,True,False,True,True,True,True,True,True,False



One-hot 编码后的数据维度: (5891, 20)
列名 (商品品类): ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '3', '4', '5', '6', '7', '8', '9']


### 3. 应用Apriori算法挖掘频繁项集

频繁项集是指那些经常一起出现的商品品类组合。Apriori算法通过迭代的方式找出这些组合。
我们需要设定一个 **最小支持度 (min_support)** 阈值。支持度是指一个项集（品类组合）在所有事务中出现的比例。只有支持度大于等于该阈值的项集才被认为是频繁的。

选择合适的 `min_support` 非常重要：
*   太高：可能导致找不到足够的频繁项集。
*   太低：可能产生大量意义不大的项集，并增加计算时间。

我们将从一个相对较高的值开始尝试，然后根据结果进行调整。

In [5]:
if 'df_onehot' in locals() and df_onehot is not None and not df_onehot.empty:
    # 设定最小支持度阈值
    # 这个值需要根据数据和期望结果进行调整。
    # 对于用户级别的事务数据，单个品类的支持度可能不会太高，组合品类的支持度会更低。
    # 我们有约5891个独立用户事务。
    # 如果一个项集出现59次，支持度约为 59/5891 = 0.01
    # 先尝试一个相对保守的值，比如0.02 (意味着项集至少在约2%的用户事务中出现)
    min_sup_threshold = 0.7
    
    print(f"开始使用Apriori算法挖掘频繁项集，最小支持度设置为: {min_sup_threshold}")
    
    frequent_itemsets = apriori(df_onehot, min_support=min_sup_threshold, use_colnames=True, verbose=1)
    # use_colnames=True: 结果中的项集会使用列名（品类ID）而不是列的索引。
    # verbose=1: 会打印出处理过程中的一些信息，如迭代次数和找到的项集数量。

    if not frequent_itemsets.empty:
        frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
        print("\n挖掘出的频繁项集 (按支持度降序排列):")
        display(frequent_itemsets.head(20)) # 显示前20个
        print(f"\n共找到 {len(frequent_itemsets)} 个频繁项集。")

        # 如果找到的频繁项集太少或太多，可以回头调整 min_sup_threshold
        if len(frequent_itemsets) < 10:
            print("警告：找到的频繁项集较少，可以尝试降低 min_support 阈值。")
        elif len(frequent_itemsets) > 1000: # 阈值可以根据实际情况调整
            print("提示：找到的频繁项集较多，如果需要更精炼的结果，可以尝试提高 min_support 阈值。")
            
    else:
        print(f"错误或未找到频繁项集。在最小支持度 {min_sup_threshold} 下没有项集满足条件。")
        print("建议尝试大幅降低 min_support 阈值，例如 0.005 或更低，然后逐步调整。")

else:
    print("错误：One-hot编码数据 (df_onehot) 不存在或为空，无法进行Apriori分析。请检查Cell 4的执行结果。")


开始使用Apriori算法挖掘频繁项集，最小支持度设置为: 0.7
Processing 4 combinations | Sampling itemset size 43

挖掘出的频繁项集 (按支持度降序排列):


,support,itemsets
0,0.97895,(1)
2,0.97623,(5)
3,0.96062,(8)
5,0.95603,"(1, 5)"
6,0.94110,"(8, 1)"
9,0.94042,"(8, 5)"
12,0.92175,"(8, 1, 5)"
1,0.72925,(2)
4,0.72178,"(2, 1)"
7,0.71872,"(2, 5)"



共找到 14 个频繁项集。


In [6]:
if 'frequent_itemsets' in locals() and frequent_itemsets is not None and not frequent_itemsets.empty:
    # 生成关联规则
    # 我们可以使用不同的指标来评估规则，最常用的是 'confidence' 和 'lift'
    # 我们先按 'confidence' (置信度) 来筛选，设置一个最小置信度阈值
    min_confidence_threshold = 0.9  # 例如，规则的置信度至少为90%
    
    print(f"\n开始从频繁项集生成关联规则，最小置信度设置为: {min_confidence_threshold}")
    
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence_threshold)
    
    if not rules.empty:
        # 对规则按 'lift' (提升度) 和 'confidence' (置信度) 降序排列，以便查看更有趣的规则
        rules = rules.sort_values(['lift', 'confidence'], ascending=[False, False])
        
        print("\n挖掘出的关联规则 (按提升度和置信度降序排列):")
        # 选择展示一些关键列，方便阅读
        # 'antecedents' -> 前项 (如果买了这些)
        # 'consequents' -> 后项 (那么可能买这些)
        # 'support' -> 规则的支持度 (前项和后项一起出现的概率)
        # 'confidence' -> 规则的置信度 (买了前项，有多大概率买后项)
        # 'lift' -> 规则的提升度 (前项的出现对后项出现的概率提升了多少)
        # 'leverage', 'conviction' 是其他衡量指标
        display(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(20)) # 显示前20条规则
        
        print(f"\n共找到 {len(rules)} 条满足条件的关联规则。")

        if len(rules) < 5 and min_confidence_threshold > 0.1: # 如果规则太少，可以提示降低置信度
             print("提示：找到的关联规则较少，可以尝试降低 min_confidence_threshold (例如 0.5 或 0.7)。")
        
    else:
        print(f"错误或未找到关联规则。在最小置信度 {min_confidence_threshold} 下没有规则满足条件。")
        print(f"频繁项集数量: {len(frequent_itemsets)}")
        print("建议：")
        print("1. 检查频繁项集是否主要为单项集。如果是，可能需要降低Apriori的 min_support 以获得更多组合项集。")
        print("2. 大幅降低 association_rules 的 min_threshold (例如设置为0.1或更低)，然后逐步调整。")

else:
    print("错误：频繁项集 (frequent_itemsets) 不存在或为空，无法生成关联规则。请检查Cell 6的执行结果和 min_support 设置。")



开始从频繁项集生成关联规则，最小置信度设置为: 0.9

挖掘出的关联规则 (按提升度和置信度降序排列):


,antecedents,consequents,support,confidence,lift
20,(2),"(8, 1)",0.70243,0.96322,1.02351
16,(2),"(1, 5)",0.71176,0.97602,1.02091
23,(2),"(8, 5)",0.70005,0.95996,1.02078
22,"(2, 5)",(8),0.70005,0.97402,1.01395
19,"(2, 1)",(8),0.70243,0.97319,1.01309
17,(2),(8),0.70922,0.97253,1.01240
18,"(8, 2)",(1),0.70243,0.99043,1.01172
15,"(2, 5)",(1),0.71176,0.99032,1.01161
21,"(8, 2)",(5),0.70005,0.98708,1.01110
12,(2),(1),0.72178,0.98976,1.01104



共找到 24 条满足条件的关联规则。


### 4. 关联规则解读与商业应用

在设定最小支持度为0.7和最小置信度为0.9的条件下，我们成功挖掘出了24条强关联规则。这些规则主要揭示了平台核心品类之间的紧密购买联系。

**关键指标回顾：**
*   **Antecedents (前项)**: 规则的左手边，代表条件项集。
*   **Consequents (后项)**: 规则的右手边，代表结果项集。
*   **Support (规则支持度)**: 前项和后项一起在所有事务中出现的比例。由于我们的频繁项集筛选基于高支持度 (0.7)，所有规则的支持度都很高。
*   **Confidence (置信度)**: 购买了前项的用户中，有多大比例也购买了后项。我们筛选的规则置信度均在0.9以上，表明关联性非常强。
*   **Lift (提升度)**: Lift(A->B) = Conf(A->B) / Support(B)。
    *   Lift > 1: 前项的出现提升了后项的购买概率（正相关）。
    *   Lift = 1: 无关联。
    *   Lift < 1: 负相关。

**规则解读与洞察：**

1.  **核心“铁三角”品类 (1, 5, 8) 的强绑定效应**：
    *   我们观察到大量涉及品类1、品类5和品类8两两组合或三者组合的规则，它们的置信度都极高（通常 > 0.94）。
        *   例如：`{1, 5} -> {8}` (conf: ~0.964)
        *   例如：`{8, 1} -> {5}` (conf: ~0.979)
        *   例如：`{8, 5} -> {1}` (conf: ~0.980)
    *   **解读**：这清晰地表明，绝大多数用户在购物时会将这三个核心品类一起纳入考虑和购买。它们是用户购物篮中最常见、最稳固的组合。
    *   **提升度分析**：虽然置信度很高，但这些核心品类之间的规则提升度普遍在1.00到1.004之间。这说明由于品类1, 5, 8本身就极其热门（独立支持度均>0.96），所以购买其中一个或两个对购买另一个的“额外提升”作用并不算非常大——因为用户本来就很有可能购买它们。然而，这并不减弱它们作为组合的强度。

2.  **品类2作为核心组合的强力补充/关联者**：
    *   规则如 ` {2} -> {8, 1}` (conf: ~0.963, lift: ~1.024) 和 ` {2} -> {1, 5}` (conf: ~0.976, lift: ~1.021) 非常引人注目。
    *   **解读**：购买了品类2的用户，有超过96%的极高概率会同时购买核心组合（如品类8和1，或品类1和5）。品类2的出现对这些核心组合的购买具有一定的提升作用（lift > 1.02），是目前看到的提升度最高的规则组之一。这表明品类2与核心品类1, 5, 8之间存在非常强的正相关关系，品类2可能是驱动用户购买更多核心品类的一个重要“引子”，或者是在购买核心品类场景下的一个高频搭配品。
    *   其他规则如 ` {2, 1} -> {8}` (conf: ~0.973) 或 ` {2, 5} -> {1}` (conf: ~0.990) 也印证了这一点。

3.  **普遍的高支持度与高置信度**：
    *   由于我们筛选的是高支持度的频繁项集，所有衍生出的规则自然也具有较高的支持度。这意味着这些购买模式在用户中非常普遍。
    *   高置信度（>0.9）则进一步确认了这些品类组合的“确定性”——买了A，就极大概率会买B。

**商业应用建议：**

1.  **强化核心品类 (1, 5, 8) 的组合营销与体验优化**：
    *   **捆绑促销/套餐设计**：鉴于品类1, 5, 8的“铁三角”关系，可以设计包含这三类商品的优惠套餐，或者在用户将其中任意两个加入购物车时，智能推荐第三个并给予小额组合优惠，以提升整体客单价和用户满足感。
    *   **一站式购物场景构建**：在线上，可以在这些核心品类的商品详情页或品类着陆页之间设置便捷的交叉引导链接。在线下，可以将这三类商品规划在相近或易于流动的购物区域。
    *   **库存与供应链协同**：确保这三个核心品类库存充足，尤其是在促销活动期间，避免因其中一个缺货而影响整个组合的销售。

2.  **利用品类2的强关联效应进行交叉销售与引流**：
    *   **精准推荐**：当用户将品类2的商品加入购物车或浏览品类2时，应优先、显著地向其推荐品类1、5、8的商品。
    *   **营销活动设计**：可以将品类2作为参与门槛，例如“购买品类2商品满XX元，即可以优惠价换购品类1/5/8的指定商品”，或者“购买品类1/5/8的商品，即可以X折购买品类2商品”。
    *   **品类2价值提升**：分析品类2的具体商品构成，如果其本身价值不高，可以思考如何通过与核心高价值品类（如品类1）的强关联来提升其“被看见”和“被顺带购买”的机会。

3.  **基于高置信度规则优化推荐系统**：
    *   虽然提升度不总是非常高，但极高的置信度本身就是推荐系统的重要依据。例如，如果规则 `A -> B` 的置信度是98%，那么在用户购买A后，推荐B的成功率会非常高。可以将这些高置信度规则直接应用于“购买此商品的用户还购买了”、“猜你喜欢”等推荐模块。

4.  **考虑“反向”规则的意义（虽然此处Lift不高，但思路可借鉴）**：
    *   例如，我们有 `(8) -> (1, 5)`。这意味着购买了品类8的用户也极大概率会买品类1和5。这可以用于在用户结账前或查看购物车时，进行“查漏补缺”式的提醒：“您可能还需要XX品类（1或5）的商品哦！”

5.  **进一步探索（如果需要更细致的规则）**：
    *   如果业务上希望找到一些非核心品类之间的、或者提升度更高的、但可能支持度和置信度略低的“惊喜”关联，可以适当**调低`min_support`和`min_confidence`阈值**，并重点关注**提升度（Lift）较高**的规则。不过，这可能会产生大量规则，需要更仔细的筛选和业务判断。以我们目前的高支持度设定，找到的强规则更多是围绕核心品类展开的。

**总结**：
本次购物篮分析成功地验证并量化了平台核心品类1、5、8之间的极强购买绑定关系，并发现了品类2作为这些核心品类强力补充者的角色。这些发现为平台在组合营销、交叉销售、商品推荐和运营策略优化方面提供了直接且有价值的参考。